# Step 1: Generate trajectories with random noise

In [1]:
from TrajectoryGenerator import TrajectoryGenerator

generator = TrajectoryGenerator(seed=1618142069)
generator.generate_trajectories(amount=2, 
                      initial_value_path="/home/opendihu/Forschungsprojekt-Clean/fibre_0_0D_0000350.py", 
                      output_folder_path="/home/opendihu/Forschungsprojekt-Clean/trajectories")

test_trajectory_generator = TrajectoryGenerator(seed=116101115116)
test_trajectory_generator.generate_trajectories(amount=2, 
                      initial_value_path="/home/opendihu/Forschungsprojekt-Clean/fibre_0_0D_0000350.py", 
                      output_folder_path="/home/opendihu/Forschungsprojekt-Clean/testTrajectories")

# Step 2: Simulate propagation of generated trajectories for 6ms with opendihu

In [2]:
from DataGenerator import DataGenerator
trainings_data_generator = DataGenerator(trajectories_folder_path="/home/opendihu/Forschungsprojekt-Clean/trajectories",
                                                 opendihu_release_folder_path="/home/opendihu/opendihu/examples/electrophysiology/fibers/multiple_fibers/build_release",
                                                 data_output_folder_path="/home/opendihu/Forschungsprojekt-Clean/trainingData",
                                                 settings_multiple_fibers_path="/home/opendihu/Forschungsprojekt-Clean/settings_multiple_fibers.py")
trainings_data_generator.generate()



test_data_generator = DataGenerator(trajectories_folder_path="/home/opendihu/Forschungsprojekt-Clean/testTrajectories",
                                                 opendihu_release_folder_path="/home/opendihu/opendihu/examples/electrophysiology/fibers/multiple_fibers/build_release",
                                                 data_output_folder_path="/home/opendihu/Forschungsprojekt-Clean/testData",
                                                 settings_multiple_fibers_path="/home/opendihu/Forschungsprojekt-Clean/settings_multiple_fibers.py")
test_data_generator.generate()

# Step 3: Calculate the errors

In [3]:
from ErrorCalculator import ErrorCalculator

error_calculator = ErrorCalculator(trainings_data_folder_path="/home/opendihu/Forschungsprojekt-Clean/trainingData")
error_calculator.calculate_all_errors()


test_error_calculator = ErrorCalculator(trainings_data_folder_path="/home/opendihu/Forschungsprojekt-Clean/testData")
test_error_calculator.calculate_all_errors()

# Step 4 Set the timesteps

In [4]:
timesteps = 63

# Step 5.1: Define the model
## more models are listed in the ModelStructures Notebook

In [11]:
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = models.Sequential()

# The first two layers with 32 filters of window size 3x3
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(timesteps, 1191, 4)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
#model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
#model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 63, 1191, 32)      1184      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 61, 1189, 32)      9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 59, 1187, 32)      9248      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 57, 1185, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 1185, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 1185, 64)      18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 1183, 64)     

# Step 5.2: Compile the model

In [6]:
from tensorflow.keras import losses
model.compile(optimizer='adam', loss=losses.MSE)

# Alternative Step 5: Load a model

In [13]:
from tensorflow import keras
model = keras.models.load_model('/home/opendihu/Forschungsprojekt-Clean/trainedCNNs/CNN0reduced.tf')

# Step 6: Train model

In [7]:
from DataLoader import DataLoader
from Engine import ModelTrainer
data_loader = DataLoader(trainings_data_folder_path="/home/opendihu/Forschungsprojekt-Clean/trainingData", timesteps=timesteps) # optional: add model_checkpoint_folder_path and/or csv_logging_file_path to arguments
model_trainer = ModelTrainer(model=model, data_loader=data_loader)
model_trainer.train(epochs=2, seed=1623157854)

Train on 32 samples, validate on 8 samples
Epoch 1/2
32/32 [==============================] - 31s 975ms/sample - loss: 2.2156 - val_loss: 1773.5680
Epoch 2/2
32/32 [==============================] - 28s 865ms/sample - loss: 1756.7784 - val_loss: 1.8098


# Step 7: (Optional) Save model

In [8]:
model_trainer.save_model("/home/opendihu/Forschungsprojekt-Clean/model.tf")

# Step 8: (Optional) Evaluate model

In [14]:
from DataLoader import DataLoader
from Engine import ModelEvaluator

test_data_loader = DataLoader(trainings_data_folder_path="/home/opendihu/Forschungsprojekt-Clean/testData", timesteps=timesteps)
model_evaluator = ModelEvaluator(model=model, data_loader=test_data_loader)
model_evaluator.evaluate()

40/40 [==============================] - 9s 219ms/sample - loss: 1.6315


# Step 9: (Optional) Predict

In [10]:
from DataLoader import DataLoader
import numpy as np

test_data_loader = DataLoader(trainings_data_folder_path="/home/opendihu/Forschungsprojekt-Clean/testData", timesteps=timesteps)
x, y = test_data_loader.get_data()

sample_number = 0

predicted_log_error = model.predict(np.array([x[sample_number]]))
predicted_error = np.power(10, predicted_log_error)

log_error = y[sample_number]
error = np.power(10, log_error)

print("predicted error:", predicted_error)
print("error:", error)

predicted error: [[0.00197187 0.03621945]]
error: [5.25970195e-05 1.91340288e+00]
